In [1]:
library(tidyverse)
library(magrittr)
library(purrr) 
library(glue) 
library(stringr) 

library(rvest) 
library(polite)
library(xml2) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract



Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding




In [2]:
all_region_fire_incident <- read_html('https://www.fireandemergency.nz/incidents-and-news/incident-reports/')

In [3]:
north_region_href <- all_region_fire_incident %>%
   html_nodes('div.incidentreport__region:nth-child(6) .region .region__link') %>%
   html_attr('href') 
north_region_href

[1] "/incidents-and-news/incident-reports/incidents?region=1&day=Monday"   
[2] "/incidents-and-news/incident-reports/incidents?region=1&day=Sunday"   
[3] "/incidents-and-news/incident-reports/incidents?region=1&day=Saturday" 
[4] "/incidents-and-news/incident-reports/incidents?region=1&day=Friday"   
[5] "/incidents-and-news/incident-reports/incidents?region=1&day=Thursday" 
[6] "/incidents-and-news/incident-reports/incidents?region=1&day=Wednesday"
[7] "/incidents-and-news/incident-reports/incidents?region=1&day=Tuesday"

In [4]:
central_region_href <- all_region_fire_incident %>%
   html_nodes('div.incidentreport__region:nth-child(7) .region .region__link') %>%
   html_attr('href') 
central_region_href

[1] "/incidents-and-news/incident-reports/incidents?region=2&day=Monday"   
[2] "/incidents-and-news/incident-reports/incidents?region=2&day=Sunday"   
[3] "/incidents-and-news/incident-reports/incidents?region=2&day=Saturday" 
[4] "/incidents-and-news/incident-reports/incidents?region=2&day=Friday"   
[5] "/incidents-and-news/incident-reports/incidents?region=2&day=Thursday" 
[6] "/incidents-and-news/incident-reports/incidents?region=2&day=Wednesday"
[7] "/incidents-and-news/incident-reports/incidents?region=2&day=Tuesday"

In [5]:
south_region_href <- all_region_fire_incident %>%
   html_nodes('div.incidentreport__region:nth-child(8) .region .region__link') %>%
   html_attr('href') 
south_region_href

[1] "/incidents-and-news/incident-reports/incidents?region=3&day=Monday"   
[2] "/incidents-and-news/incident-reports/incidents?region=3&day=Sunday"   
[3] "/incidents-and-news/incident-reports/incidents?region=3&day=Saturday" 
[4] "/incidents-and-news/incident-reports/incidents?region=3&day=Friday"   
[5] "/incidents-and-news/incident-reports/incidents?region=3&day=Thursday" 
[6] "/incidents-and-news/incident-reports/incidents?region=3&day=Wednesday"
[7] "/incidents-and-news/incident-reports/incidents?region=3&day=Tuesday"

In [6]:
all_region_href <- tibble(North = north_region_href, Central = central_region_href, South = south_region_href)
all_region_href

North,Central,South
<chr>,<chr>,<chr>
/incidents-and-news/incident-reports/incidents?region=1&day=Monday,/incidents-and-news/incident-reports/incidents?region=2&day=Monday,/incidents-and-news/incident-reports/incidents?region=3&day=Monday
/incidents-and-news/incident-reports/incidents?region=1&day=Sunday,/incidents-and-news/incident-reports/incidents?region=2&day=Sunday,/incidents-and-news/incident-reports/incidents?region=3&day=Sunday
/incidents-and-news/incident-reports/incidents?region=1&day=Saturday,/incidents-and-news/incident-reports/incidents?region=2&day=Saturday,/incidents-and-news/incident-reports/incidents?region=3&day=Saturday
/incidents-and-news/incident-reports/incidents?region=1&day=Friday,/incidents-and-news/incident-reports/incidents?region=2&day=Friday,/incidents-and-news/incident-reports/incidents?region=3&day=Friday
/incidents-and-news/incident-reports/incidents?region=1&day=Thursday,/incidents-and-news/incident-reports/incidents?region=2&day=Thursday,/incidents-and-news/incident-reports/incidents?region=3&day=Thursday
/incidents-and-news/incident-reports/incidents?region=1&day=Wednesday,/incidents-and-news/incident-reports/incidents?region=2&day=Wednesday,/incidents-and-news/incident-reports/incidents?region=3&day=Wednesday
/incidents-and-news/incident-reports/incidents?region=1&day=Tuesday,/incidents-and-news/incident-reports/incidents?region=2&day=Tuesday,/incidents-and-news/incident-reports/incidents?region=3&day=Tuesday


In [7]:
install.packages('xts')
library(xts)

Installing package into ‘/home/christopher/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘xts’


The following objects are masked from ‘package:dplyr’:

    first, last




In [15]:
region_incident_data <- function(href){
    
    region_data <- glue('https://www.fireandemergency.nz{href}#')%>%
                   read_html() %>%
                   html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(2) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                   html_text2()

    region_round_time <- strptime(region_data, "%d/%m/%Y %H:%M:%S")

    for(i in 1:length(region_round_time)){
        region_round_time[i] <- align.time(region_round_time[i], n=60*30
        )

    }
    
    return(region_round_time)
    
    }
    result <- region_incident_data("/incidents-and-news/incident-reports/incidents?region=1&day=Monday#")
    result
                  

 [1] "2022-10-10 01:00:00 NZDT" "2022-10-10 05:00:00 NZDT"
 [3] "2022-10-10 07:00:00 NZDT" "2022-10-10 07:30:00 NZDT"
 [5] "2022-10-10 07:30:00 NZDT" "2022-10-10 07:30:00 NZDT"
 [7] "2022-10-10 08:00:00 NZDT" "2022-10-10 08:00:00 NZDT"
 [9] "2022-10-10 08:00:00 NZDT" "2022-10-10 08:00:00 NZDT"
[11] "2022-10-10 08:30:00 NZDT" "2022-10-10 08:30:00 NZDT"
[13] "2022-10-10 08:30:00 NZDT" "2022-10-10 09:00:00 NZDT"
[15] "2022-10-10 09:00:00 NZDT" "2022-10-10 09:00:00 NZDT"
[17] "2022-10-10 09:30:00 NZDT" "2022-10-10 09:30:00 NZDT"
[19] "2022-10-10 09:30:00 NZDT" "2022-10-10 09:30:00 NZDT"
[21] "2022-10-10 10:00:00 NZDT" "2022-10-10 10:00:00 NZDT"
[23] "2022-10-10 10:00:00 NZDT" "2022-10-10 10:30:00 NZDT"
[25] "2022-10-10 10:30:00 NZDT"

In [16]:
region_incident_data <- function(id, day){
    
    region_href <- glue('https://www.fireandemergency.nz/incidents-and-news/incident-reports/incidents?region={id}&day={day}#')%>%
                   read_html() 
    
    Incident <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(1) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
    
    Date_and_Time <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(2) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()

    Date_and_Time <- strptime(Date_and_Time, "%d/%m/%Y %H:%M:%S")

    for(i in 1:length(Date_and_Time)){
        Date_and_Time[i] <- align.time(Date_and_Time[i], n=60*30
        )

    }
        
    Location <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(3) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
    
    Duration <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(4) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
    
    Station <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(5) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
    
    Result <- region_href %>%
                  html_nodes(glue('.report__table .report__table__body .report__table__row:nth-child(6) .report__table__cell+ .report__table__cell--value p:nth-child(1)')) %>%
                  html_text2()
    
    region_incident_table <- tibble(Incident_no = Incident, Date_and_Time = Date_and_Time, Location = Location, Duration = Duration, Station = Station, Result = Result)
    
    return(region_incident_table)
    
    }

format:
{region}_{day}_data

### North fire incident data (Monday to Sunday)

In [17]:
north_monday_data <- region_incident_data("1", "Monday")
north_monday_data

Incident_no,Date_and_Time,Location,Duration,Station,Result
<chr>,<dttm>,<chr>,<chr>,<chr>,<chr>
F3575491,2022-10-10 01:00:00,MAEROA HAMILTON CITY,00:00:00,Te Rapa,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MED)"
F3575502,2022-10-10 05:00:00,KAIAUA HAURAKI DISTRICT,00:00:00,Kaiaua,9900: NON-FIRE EVENT: EXCL RESCUE/FALSE ALARM (MIN)
F3575529,2022-10-10 07:00:00,PAPAKURA AUCKLAND,00:08:09,Papakura,7100: FALSE ALARM (STRU)
F3575533,2022-10-10 07:30:00,TITIRANGI AUCKLAND,00:09:38,Titirangi,7100: FALSE ALARM (STRU)
F3575543,2022-10-10 07:30:00,PAPATOETOE AUCKLAND,00:00:00,Papatoetoe,"3100: RESCUE,EMERGENCY,MEDICAL CALL (RESC)"
F3575544,2022-10-10 07:30:00,REMUERA AUCKLAND,00:00:00,Remuera,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MED)"
F3575545,2022-10-10 08:00:00,PARNELL AUCKLAND,00:11:23,Parnell,7100: FALSE ALARM (STRU)
F3575546,2022-10-10 08:00:00,TE AWAMUTU WAIPA DISTRICT,00:00:00,Te Awamutu,1500: MISCELLANEOUS FIRE (STRU)
F3575550,2022-10-10 08:00:00,ONERAHI WHANGAREI DISTRICT,00:00:00,Onerahi,9900: NON-FIRE EVENT: EXCL RESCUE/FALSE ALARM (MIN)


In [10]:
north_tuesday_data <- region_incident_data("1", "Tuesday")

In [11]:
north_wednesday_data <- region_incident_data("1", "Wednesday")

In [12]:
north_thursday_data <- region_incident_data("1", "Thursday")

In [13]:
north_friday_data <- region_incident_data("1", "Friday")

In [14]:
north_saturday_data <- region_incident_data("1", "Saturday")

In [15]:
north_sunday_data <- region_incident_data("1", "Sunday")
north_sunday_data

Incident_no,Date_and_Time,Location,Duration,Station,Result
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
F3570958,02/10/2022 00:02:48,PAKURANGA AUCKLAND,00:16:16,Mount Wellington,7100: FALSE ALARM (SPRNKLR)
F3570962,02/10/2022 00:15:30,WAIUKU AUCKLAND,00:19:35,Waiuku,9900: NON-FIRE EVENT: EXCL RESCUE/FALSE ALARM (MIN)
F3570963,02/10/2022 00:18:59,PAPATOETOE AUCKLAND,00:00:00,Papatoetoe,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MED)"
F3570967,02/10/2022 00:24:54,WAIRAU VALLEY AUCKLAND,00:15:11,Takapuna,7100: FALSE ALARM (SPRNKLR)
F3570974,02/10/2022 00:45:03,TE ATATU PENINSULA AUCKLAND,00:13:07,Te Atatu,7100: FALSE ALARM (SPRNKLR)
F3570975,02/10/2022 00:46:46,REGENT WHANGAREI DISTRICT,00:00:00,Whangarei,1500: MISCELLANEOUS FIRE (MIN)
F3570979,02/10/2022 00:57:24,HAMILTON EAST HAMILTON CITY,00:00:00,Hamilton,9900: NON-FIRE EVENT: EXCL RESCUE/FALSE ALARM (MIN)
F3570980,02/10/2022 01:01:37,EPSOM AUCKLAND,00:14:07,Te Atatu,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MED)"
F3570982,02/10/2022 01:02:12,TE ATATU PENINSULA AUCKLAND,00:00:00,Te Atatu,9900: NON-FIRE EVENT: EXCL RESCUE/FALSE ALARM (MIN)


### Central fire incident data (Monday to Sunday)

In [16]:
central_monday_data <- region_incident_data("2", "Monday")

In [17]:
central_tuesday_data <- region_incident_data("2", "Tuesday")

In [18]:
central_wednesday_data <- region_incident_data("2", "Wednesday")

In [19]:
central_thursday_data <- region_incident_data("2", "Thursday")

In [20]:
central_friday_data <- region_incident_data("2", "Friday")

In [21]:
central_saturday_data <- region_incident_data("2", "Saturday")

In [22]:
central_sunday_data <- region_incident_data("2", "Sunday")

### South fire incident data (Monday to Sunday)

In [23]:
south_monday_data <- region_incident_data("3", "Monday")

In [24]:
south_tuesday_data <- region_incident_data("3", "Tuesday")

In [25]:
south_wednesday_data <- region_incident_data("3", "Wednesday")

In [26]:
south_thursday_data <- region_incident_data("3", "Thursday")

In [27]:
south_friday_data <- region_incident_data("3", "Friday")

In [28]:
south_saturday_data <- region_incident_data("3", "Saturday")

In [29]:
south_sunday_data <- region_incident_data("3", "Sunday")

## Merge all dataframe from Monday to Sunday into sngle one data frame

In [19]:
get_week_data <- function(region) {
    result_tibble <- tibble(
        Incident_no = character(),
        Date_and_Time = date(),
        Location = character(),
        Duration= character(),
        Station = character(),
        Result = character()
    )
    temp <- c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")
    for(i in temp){
        result_tibble <- rbind(result_tibble, region_incident_data(region, i))
    }
    
    return(result_tibble)
}

the_tibble <- get_week_data("2")
the_tibble

Incident_no,Date_and_Time,Location,Duration,Station,Result
<chr>,<dttm>,<chr>,<chr>,<chr>,<chr>
F3575487,2022-10-10 00:30:00,OPUNAKE SOUTH TARANAKI DISTRICT,00:00:00,Opunake,"3100: RESCUE,EMERGENCY,MEDICAL CALL (RESC)"
F3575492,2022-10-10 01:30:00,HIGHBURY PALMERSTON NORTH CITY,00:30:40,Palmerston North,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MED)"
F3575505,2022-10-10 05:00:00,INGLEWOOD NEW PLYMOUTH DISTRICT,00:00:00,Inglewood,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MIN)"
F3575508,2022-10-10 05:30:00,LONGBURN PALMERSTON NORTH CITY,04:23:10,Palmerston North,
F3575511,2022-10-10 05:30:00,MAREWA NAPIER CITY,00:00:00,Napier,1200: MOBILE PROPERTY FIRE (STRU)
F3575513,2022-10-10 05:30:00,WHANGANUI EAST WHANGANUI DISTRICT,00:55:36,Whanganui,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MED)"
F3575519,2022-10-10 06:00:00,MARAENUI NAPIER CITY,00:00:00,Napier,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MED)"
F3575520,2022-10-10 06:30:00,HUTT CENTRAL LOWER HUTT CITY,00:18:15,Seaview,7100: FALSE ALARM (STRU)
F3575534,2022-10-10 07:30:00,TARADALE NAPIER CITY,00:20:52,Napier,7100: FALSE ALARM (STRU)


## Seperate date and time columns. Also adding the new column for the day of the week

In [28]:
library(lubridate)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [30]:
# get_date <- function(datetime) {
#     the_time <- strsplit(datetime, split=" ")
#     return(the_time[[1]][2])
# }
# the_tibble
the_tibble <- the_tibble %>% mutate(Date = date(Date_and_Time)) %>% mutate(Time = format(Date, format = "%H:%M:%S"))
the_tibble

Incident_no,Date_and_Time,Location,Duration,Station,Result,Date,Time
<chr>,<dttm>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>
F3575487,2022-10-10 00:30:00,OPUNAKE SOUTH TARANAKI DISTRICT,00:00:00,Opunake,"3100: RESCUE,EMERGENCY,MEDICAL CALL (RESC)",2022-10-10,00:00:00
F3575492,2022-10-10 01:30:00,HIGHBURY PALMERSTON NORTH CITY,00:30:40,Palmerston North,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MED)",2022-10-10,00:00:00
F3575505,2022-10-10 05:00:00,INGLEWOOD NEW PLYMOUTH DISTRICT,00:00:00,Inglewood,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MIN)",2022-10-10,00:00:00
F3575508,2022-10-10 05:30:00,LONGBURN PALMERSTON NORTH CITY,04:23:10,Palmerston North,,2022-10-10,00:00:00
F3575511,2022-10-10 05:30:00,MAREWA NAPIER CITY,00:00:00,Napier,1200: MOBILE PROPERTY FIRE (STRU),2022-10-10,00:00:00
F3575513,2022-10-10 05:30:00,WHANGANUI EAST WHANGANUI DISTRICT,00:55:36,Whanganui,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MED)",2022-10-10,00:00:00
F3575519,2022-10-10 06:00:00,MARAENUI NAPIER CITY,00:00:00,Napier,"3100: RESCUE,EMERGENCY,MEDICAL CALL (MED)",2022-10-10,00:00:00
F3575520,2022-10-10 06:30:00,HUTT CENTRAL LOWER HUTT CITY,00:18:15,Seaview,7100: FALSE ALARM (STRU),2022-10-10,00:00:00
F3575534,2022-10-10 07:30:00,TARADALE NAPIER CITY,00:20:52,Napier,7100: FALSE ALARM (STRU),2022-10-10,00:00:00


## Seperate the result code and the sesult description into 2 columns

In [46]:
get_result_code <- function(result){
    splitted_result <- strsplit(result, split=":")
    result_list <- vector(mode = "list", len = 0)
    for(i in splitted_result){
        # print(i[1])
        result_list<- append(result_list, i[1])
    }
    return(result_list)
}

get_result_desc <- function(result){
    splitted_result <- strsplit(result, split=":")
    desc_list <- vector(mode = "list", len = 0)
    for(i in splitted_result){
        desc = ""
        for(ind in seq_along(i)){

            if(ind != 1) {
                desc <- glue(desc,i[[ind]])
            }
        }

        desc_list <- append(desc_list, desc)
    }
    
    return(desc_list)
}


the_tibble <- the_tibble %>% mutate(Result_code = get_result_code(Result)) %>% mutate(Result_Description = get_result_desc(Result)) %>% mutate(across(Result_Description, as.character)) 
the_tibble




Incident_no,Date_and_Time,Location,Duration,Station,Result,Date,Time,Day,Result_code,Result_Description
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>,<ord>,<list>,<chr>
F3571388,03/10/2022 00:33:43,KILBIRNIE WELLINGTON CITY,00:14:28,Kilbirnie,7100: FALSE ALARM (STRU),3-10-20,00:33:43,Mon,7100,FALSE ALARM (STRU)
F3571408,03/10/2022 03:36:39,WAITANGIRUA PORIRUA CITY,00:00:00,Porirua,9900: NON-FIRE EVENT: EXCL RESCUE/FALSE ALARM (MIN),3-10-20,00:33:43,Mon,9900,NON-FIRE EVENT EXCL RESCUE/FALSE ALARM (MIN)
F3571414,03/10/2022 04:36:39,CAMBERLEY HASTINGS DISTRICT,00:00:00,Hastings,9900: NON-FIRE EVENT: EXCL RESCUE/FALSE ALARM (MIN),3-10-20,00:33:43,Mon,9900,NON-FIRE EVENT EXCL RESCUE/FALSE ALARM (MIN)
F3571416,03/10/2022 05:15:05,PALMERSTON NORTH PALMERSTON NORTH CITY,00:24:33,Palmerston North,7100: FALSE ALARM (STRU),3-10-20,00:33:43,Mon,7100,FALSE ALARM (STRU)
F3571421,03/10/2022 05:44:28,EGMONT VILLAGE NEW PLYMOUTH DISTRICT,00:00:00,Inglewood,"3100: RESCUE,EMERGENCY,MEDICAL CALL (RESC)",3-10-20,00:33:43,Mon,3100,"RESCUE,EMERGENCY,MEDICAL CALL (RESC)"
F3571437,03/10/2022 07:35:41,MERRILANDS NEW PLYMOUTH DISTRICT,00:00:00,New Plymouth,9900: NON-FIRE EVENT: EXCL RESCUE/FALSE ALARM (MIN),3-10-20,00:33:43,Mon,9900,NON-FIRE EVENT EXCL RESCUE/FALSE ALARM (MIN)
F3571438,03/10/2022 07:41:54,WHITBY PORIRUA CITY,00:00:00,Porirua,"3100: RESCUE,EMERGENCY,MEDICAL CALL (RESC)",3-10-20,00:33:43,Mon,3100,"RESCUE,EMERGENCY,MEDICAL CALL (RESC)"
F3571439,03/10/2022 07:41:31,LONGBURN PALMERSTON NORTH CITY,00:00:00,Palmerston North,7100: FALSE ALARM (STRU),3-10-20,00:33:43,Mon,7100,FALSE ALARM (STRU)
F3571460,03/10/2022 08:20:18,MAHOE STRATFORD DISTRICT,00:00:00,Kaponga,1200: MOBILE PROPERTY FIRE (STRU),3-10-20,00:33:43,Mon,1200,MOBILE PROPERTY FIRE (STRU)


## rounded the time to 30 minutes

In [47]:
the_tibble %>% mutate(Rounded_time = round_date(Date_and_Time, unit="30 mins"))

Warning message:
“All formats failed to parse. No formats found.”


ERROR: [1m[33mError[39m in [1m[1m`mutate()`:[22m
[1m[22m[33m![39m Problem while computing `Rounded_time = round_date(Date_and_Time, unit
  = "30 mins")`.
[1mCaused by error in [1m[1m`UseMethod()`:[22m
[33m![39m no applicable method for 'reclass_date' applied to an object of class "character"
